# 厚壁压力容器计算

In [1]:
%reset -f
import numpy as np
from numpy.linalg.linalg import solve 

def sfa (ri, rm, ro, Ei, vi, Eo, vo, itf=0, pi=0, po=0):
    ''' 
    这个函数用来计算过盈配合
    
    ri, rm, ro : 内径, 中径， 外径
    Ei, vi：内零件杨氏模量与泊松比
    Eo, vo: 外零件杨氏模量与泊松比
    itf: 可选参数, 过盈尺寸
    pi, po: 可选参数, 内外压力, 默认为零
   ------------------------------- 输出结果数量级为10^6
    '''
    if ri == 0:
        if itf == 0: ## 需要计算 itf，pi值需要输入既为配合界面压强
            A = np.array([[1, 0, 0],
                          [0, 1, -1/rm**2],
                          [0, 1, -1/ro**2]])
            b = np.array([[-pi, -pi, -po]]).T
            x = solve(A, b)  # [A, C, D]
            itf = np.sum(np.array([[-(1-vi)/Ei, (1-vo)/Eo, (1+vo)/(Eo*rm**2)]]).T * x) * rm * 10**3
            print(f'过盈尺寸={itf}mm\n')
            x = np.insert(x, 1, [0], axis=0)
        else: ## 给出 itf 需要计算配合压强 pi
            A = np.array([[0, 1, -1/ro**2],
                          [1, -1, 1/rm**2],
                          [-1*(1-vi)/Ei, (1-vo)/Eo, (1+vo)/(Eo*rm**2)]])
            b = np.array([[-po, 0, itf/rm]]).T
            x = solve(A,b)
            print(f'界面与内部压强={x[0] / 1e6}MPa\n')
            x = np.insert(x, 1, [0], axis=0)
    else:
        A = np.array([[1, -1/ri**2, 0, 0], 
                      [0, 0, 1, -1/ro**2],
                      [1, -1/rm**2, -1, 1/rm**2],
                      [-1*(1-vi)/Ei, -(1+vi)/(Ei*rm**2), (1-vo)/Eo, (1+vo)/(Eo*rm**2)]])
        b = np.array([[-pi, -po, 0, itf/rm]]).T
        x = solve(A,b)

    if ri==0:
        stress_h_ri = x[0]
    else:
        stress_h_ri = x[0] + x[1] / ri**2
        
    stress_r_rm = x[0] - x[1] / rm**2
    stress_h_rm_i = x[0] + x[1] / rm**2
    stress_h_rm_o = x[2] + x[3] / rm**2
    stress_h_ro = x[2] + x[3] / ro**2
    
    if stress_h_ri > stress_h_rm_o:
        shm = stress_h_ri
        sr_ = -pi
        rc = 'ri'
    elif stress_h_ri < stress_h_rm_o:
        shm = stress_h_rm_o
        sr_ = stress_r_rm
        rc = 'rm'
    sz = 0 # z方向应力未指定 默认为0
    VonStress = np.sqrt(((shm - sr_)**2 + (sr_ - sz)**2 + (sz - shm)**2) / 2)
    Tresca = shm - sr_
        
    print(f'系数为\n{x / 1e6}10^6\n')
    print(f'内环 内径处 径向和环向压强分别为 {-pi / 1e6}MPa, {stress_h_ri / 1e6}MPa\n')
    print(f'配合处 径向 配和压强为 {stress_r_rm / 1e6}MPa\n')
    print(f'配合处 内外 环向 压强分别为 {stress_h_rm_i / 1e6}MPa, {stress_h_rm_o / 1e6}MPa\n')
    print(f'外环 外径处 径向和环向压强分别为 {-po / 1e6}MPa, {stress_h_ro / 1e6}MPa\n')
    print(f'最大环向应力 = {shm / 1e6}MPa, 对应位置为 {rc} (危险点)')
    print(f'Von Mises Criteria = {VonStress / 1e6}MPa')
    print(f'Tresca Criteria = {Tresca / 1e6}MPa')
    
    return x


In [3]:
A, B, C, D = sfa(ri=0.05/2, rm=0.06/2, ro=0.07/2, Ei=3e9, vi=0.32, Eo=3e9, vo=0.32, itf=0.02e-3, pi=20e5, po=0e6)

系数为
[[1.54166667e+00]
 [2.21354167e-03]
 [2.54166667e+00]
 [3.11354167e-03]]10^6

内环 内径处 径向和环向压强分别为 -2.0MPa, [5.08333333]MPa

配合处 径向 配和压强为 [-0.91782407]MPa

配合处 内外 环向 压强分别为 [4.00115741]MPa, [6.00115741]MPa

外环 外径处 径向和环向压强分别为 -0.0MPa, [5.08333333]MPa

最大环向应力 = [6.00115741]MPa, 对应位置为 rm (危险点)
Von Mises Criteria = [6.50878622]MPa
Tresca Criteria = [6.91898148]MPa
